# Models 

In this section we’ll take a closer look at creating and using a model. We’ll use the AutoModel class, which is handy when you want to instantiate any model from a checkpoint.

The AutoModel class and all of its relatives are actually simple wrappers over the wide variety of models available in the library. It’s a clever wrapper as it can automatically guess the appropriate model architecture for your checkpoint, and then instantiates a model with this architecture.

However, if you know the type of model you want to use, you can use the class that defines its architecture directly. Let’s take a look at how this works with a BERT model.

## Creating a Transformer


The first thing we’ll need to do to initialize a BERT model is load a configuration object:



In [1]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)

In [2]:
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.7.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [3]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


As you saw earlier, we could replace BertModel with the equivalent AutoModel class. We’ll do this from now on as this produces checkpoint-agnostic code; if your code works for one checkpoint, it should work seamlessly with another. This applies even if the architecture is different, as long as the checkpoint was trained for a similar task (for example, a sentiment analysis task).

In the code sample above we didn’t use BertConfig, and instead loaded a pretrained model via the bert-base-cased identifier. This is a model checkpoint that was trained by the authors of BERT themselves; you can find more details about it in its model card.

This model is now initialized with all the weights of the checkpoint. It can be used directly for inference on the tasks it was trained on, and it can also be fine-tuned on a new task. By training with pretrained weights rather than from scratch, we can quickly achieve good results.

The weights have been downloaded and cached (so future calls to the from_pretrained method won’t re-download them) in the cache folder, which defaults to ~/.cache/huggingface/transformers. You can customize your cache folder by setting the HF_HOME environment variable.

The identifier used to load the model can be the identifier of any model on the Model Hub, as long as it is compatible with the BERT architecture. The entire list of available BERT checkpoints can be found here.



## Saving methods


Saving a model is as easy as loading one — we use the save_pretrained method, which is analogous to the from_pretrained method:

In [5]:
model.save_pretrained("directory_on_my_computer")


In [6]:
!ls directory_on_my_computer

config.json  pytorch_model.bin


If you take a look at the config.json file, you’ll recognize the attributes necessary to build the model architecture. This file also contains some metadata, such as where the checkpoint originated and what 🤗 Transformers version you were using when you last saved the checkpoint.



The pytorch_model.bin file is known as the state dictionary; it contains all your model’s weights. The two files go hand in hand; the configuration is necessary to know your model’s architecture, while the model weights are your model’s parameters.



## Using a Transformer model for inference


Now that you know how to load and save a model, let’s try using it to make some predictions. Transformer models can only process numbers — numbers that the tokenizer generates. But before we discuss tokenizers, let’s explore what inputs the model accepts.

Tokenizers can take care of casting the inputs to the appropriate framework’s tensors, but to help you understand what’s going on, we’ll take a quick look at what must be done before sending the inputs to the model.

Let’s say we have a couple of sequences:

In [7]:
sequences = [
  "Hello!",
  "Cool.",
  "Nice!"
]

In [8]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The tokenizer converts these to vocabulary indices which are typically called input IDs. Each sequence is now a list of numbers! The resulting output is:



In [16]:
encoded_sequences = tokenizer(sequences, padding=True, truncation=True, return_tensors="np")['input_ids'].tolist()

This is a list of encoded sequences: a list of lists. Tensors only accept rectangular shapes (think matrices). This “array” is already of rectangular shape, so converting it to a tensor is easy:



In [17]:
encoded_sequences

[[101, 7592, 999, 102], [101, 4658, 1012, 102], [101, 3835, 999, 102]]

In [22]:
import torch

model_inputs= torch.tensor(encoded_sequences)

## Using the tensors as inputs to the model

Making use of the tensors with the model is extremely simple — we just call the model with the inputs:


In [23]:
output = model(model_inputs)

While the model accepts a lot of different arguments, only the input IDs are necessary. We’ll explain what the other arguments do and when they are required later, but first we need to take a closer look at the tokenizers that build the inputs that a Transformer model can understand.

In [34]:
output.last_hidden_state.detach().numpy()

array([[[ 4.4495761e-01,  4.8276371e-01,  2.7797160e-01, ...,
         -5.4033030e-02,  3.9393550e-01, -9.4770476e-02],
        [ 2.4942905e-01, -4.4092986e-01,  8.1772316e-01, ...,
         -3.1916568e-01,  2.2992224e-01, -4.1171860e-02],
        [ 1.3667573e-01,  2.2517854e-01,  1.4501970e-01, ...,
         -4.6914764e-02,  2.8224215e-01,  7.5565711e-02],
        [ 1.1788858e+00,  1.6738428e-01, -1.8187094e-01, ...,
          2.4671389e-01,  1.0440769e+00, -6.1963657e-03]],

       [[ 3.6435887e-01,  3.2464445e-02,  2.0257673e-01, ...,
          6.0110606e-02,  3.2451311e-01, -2.0995857e-02],
        [ 7.1865958e-01, -4.8725188e-01,  5.1740414e-01, ...,
         -4.4012007e-01,  1.4553043e-01, -3.7544787e-02],
        [ 3.3223268e-01, -2.3270920e-01,  9.4876043e-02, ...,
         -2.5268158e-01,  3.2171968e-01,  8.1144931e-04],
        [ 1.2523224e+00,  3.5754374e-01, -5.1319774e-02, ...,
         -3.7839824e-01,  1.0526477e+00, -5.6254929e-01]],

       [[ 2.4042279e-01,  1.4717807e